# Assignment 1: Yield Curves

## 2.1 Fundamental Questions

### Q2: Selecting 10 Bonds to Construct Yield and Spot Curves

Canadian Government bond data including ISIN, coupon, yield, maturity date, issue date, issue price and historical time series close prices were collected from the following two links: 

* https://markets.businessinsider.com/bonds/finder?borrower=71&maturity=shortterm&yield=&bondtype=2%2c3%2c4%2c16&coupon=&currency=184&rating=&country=19
* https://markets.businessinsider.com/bonds/finder?borrower=71&maturity=midterm&yield=&bondtype=2%2c3%2c4%2c16&coupon=&currency=184&rating=&country=19

This is done through web scraping via the `Selenium` Python package and will then be repackaged into a dataframe and saved as a .csv file.

#### Pulling data for all bonds

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import datetime as dt
import pandas as pd
import numpy as np
import time

from tqdm import tqdm

In [2]:
option = Options()
option.headless = False
driver = webdriver.Chrome(options=option)
bond_dict = {}
start_urls = ['https://markets.businessinsider.com/bonds/finder?borrower=71&maturity=shortterm&yield=&bondtype=2%2/c3%2/c4%2/c16&coupon=&currency=184&rating=&country=19',
               'https://markets.businessinsider.com/bonds/finder?p=2&borrower=71&maturity=shortterm&bondtype=2%/2c3%/2c4%2/c16&currency=184&country=19',
               'https://markets.businessinsider.com/bonds/finder?borrower=71&maturity=midterm&yield=&bondtype=2%2/c3%2/c4%2/c16&coupon=&currency=184&rating=&country=19']
count = 1

# collecting bond urls, coupons, yields and maturity date from starting urls
# individual bond urls are needed to collect historical time series data
for url in start_urls:
    driver.get(url)
    table = driver.find_element(By.XPATH, 
                                    '//*[@id="bond-searchresults-container"]/div/div/div[1]/div[1]/div[2]/div/div/div/table/tbody')
    rows = table.find_elements(By.TAG_NAME, 'tr')

    for row in rows:
        link = row.find_elements(By.TAG_NAME, 'a')[0].get_attribute('href')
        coupon = row.find_elements(By.TAG_NAME, 'td')[2].text
        yield_ = row.find_elements(By.TAG_NAME, 'td')[3].text
        maturityDate = row.find_elements(By.TAG_NAME, 'td')[5].text

        bond_dict[count] = {'link' : link,
                           'coupon' : coupon,
                           'yield' : yield_,
                           'maturityDate' : maturityDate}
        count += 1

In [6]:
# collecting historical time series data, ISIN, name, issue price, issue date 
c = 1
for key, items in tqdm(bond_dict.items()):
    url = bond_dict[key]['link']
    driver = webdriver.Chrome(options=option)
    driver.get(url)

    time.sleep(20)
    change_view_button = WebDriverWait(driver,10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="DetailChart"]/app-root/div[2]/div/div[1]/div/div[1]/div[2]/div/div/div/div/div[3]')))
    change_view_button.click()
    time.sleep(10)

    chart = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="DetailChart"]/app-root/div[2]/app-detail-chart/div[2]/div[13]/div')))
    loc = chart.location
    size = chart.size

    bond_dict[key]['ISIN'] = driver.find_element(By.XPATH, 
                               '/html/body/main/div/div[4]/div[2]/div/div/div[1]/div[2]/div/div/div/table/tbody/tr[1]/td[2]').text
    bond_dict[key]['name'] = driver.find_element(By.XPATH, 
                               '/html/body/main/div/div[4]/div[2]/div/div/div[1]/div[2]/div/div/div/table/tbody/tr[2]/td[2]').text
    bond_dict[key]['issuePrice'] = driver.find_element(By.XPATH, 
                                     '/html/body/main/div/div[4]/div[2]/div/div/div[1]/div[2]/div/div/div/table/tbody/tr[8]/td[2]').text
    bond_dict[key]['issueDate'] = driver.find_element(By.XPATH, 
                                     '/html/body/main/div/div[4]/div[2]/div/div/div[1]/div[2]/div/div/div/table/tbody/tr[9]/td[2]').text
    
    action = webdriver.ActionChains(driver)
    action.move_to_element(chart).perform()
    action.move_by_offset(240,0).perform()

    close_ts = {}
    pace = -20
    while True:
        tooltipdate = driver.find_element(By.XPATH,'//*[@id="DetailChart"]/app-root/div[2]/app-detail-chart/div[2]/stx-hu-tooltip/stx-hu-tooltip-field[1]').text[5:9].strip()
        tooltipclose = driver.find_element(By.XPATH,'//*[@id="DetailChart"]/app-root/div[2]/app-detail-chart/div[2]/stx-hu-tooltip/stx-hu-tooltip-field[2]').text[6:]

        if tooltipdate:
            close_ts[tooltipdate] = tooltipclose
        else:
            if len(close_ts) < 10:
                print(f"{bond_dict[key]['name']} does not have sufficient ts data.")
            break

        try:
            action.move_by_offset(pace, 0).perform()
        except:
            print(f"{bond_dict[key]['name']}'s data has been scraped.")
            break
    
        time.sleep(2)
    
    bond_dict[key]['closeTS'] = close_ts
    driver.close()
    
    c += 1

 93%|███████████████████████████████████████████████████████▋    | 39/42 [1:26:09<06:30, 130.26s/it]

CANADA 23/26 does not have sufficient ts data.


100%|████████████████████████████████████████████████████████████| 42/42 [1:32:02<00:00, 131.48s/it]


#### Data Cleaning

In [61]:
df = pd.DataFrame.from_dict(bond_dict, orient='index')
df.drop(['link', 'name'], axis=1, inplace=True)
df = pd.concat([df.drop(['closeTS'], axis=1), df['closeTS'].apply(pd.Series)], axis=1)

for c in df.columns:
    if c[-1] == '/':
        df = df.rename({c : c[:-1]}, axis=1)

df.coupon = df.coupon.apply(str)
df['yield'] = df['yield'].astype(str)

for c in ['coupon', 'yield']:
    df[c] = df[c].apply(lambda x: x[:-1])

df.maturityDate = pd.to_datetime(df.maturityDate)
df.issueDate = pd.to_datetime(df.issueDate)

prefix = 'CAN'
newNames = []

for i in range(df.shape[0]):
    d = df.iloc[i]['maturityDate'].strftime('%b %y')
    c = df.iloc[i]['coupon'][:4]
    newName = str(prefix + ' ' + c + ' ' + d)
    newNames.append(newName)

df.index = newNames
    
for c in ['coupon', 'yield', 'issuePrice', '2/9', '2/8', '2/7', '2/6', '2/3', '2/2', '2/1', '1/31',
       '1/30', '1/27', '1/26', '1/25', '1/24', '1/23', '1/20', '1/19', '1/18',
       '1/17', '1/16', '1/13', '1/12', '1/11', '1/10']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df = df.round(2)  
df.head()

,coupon,yield,maturityDate,ISIN,issuePrice,issueDate,2/9,2/8,2/7,2/6,...,1/23,1/20,1/19,1/18,1/17,1/16,1/13,1/12,1/11,1/10
CAN 1.50 Jun 23,1.50,4.55,2023-06-01,CA135087A610,97.98,2012-07-30,99.11,99.09,99.07,99.06,...,98.96,98.95,98.95,98.94,98.92,98.92,98.91,98.91,98.88,98.86
CAN 2.50 Jun 24,2.50,4.33,2024-06-01,CA135087B451,98.99,2013-07-02,97.77,97.72,97.70,97.74,...,98.02,98.05,98.11,98.17,98.07,98.08,98.04,97.97,97.92,97.83
CAN 2.25 Jun 25,2.25,3.68,2025-06-01,CA135087D507,98.85,2014-06-30,96.98,96.88,96.86,96.94,...,97.43,97.51,97.68,97.70,97.45,97.45,97.26,97.05,96.92,96.74
CAN 1.75 Mar 23,1.75,NaN,2023-03-01,CA135087H490,99.56,2017-10-06,99.89,99.86,99.86,99.85,...,99.75,99.74,99.72,99.72,99.70,99.71,99.70,99.68,99.67,99.64
CAN 2.25 Mar 24,2.25,4.49,2024-03-01,CA135087J546,98.99,2018-10-05,97.77,97.74,97.70,97.72,...,97.86,97.87,97.92,97.95,97.87,97.93,97.91,97.87,97.85,97.77


In [62]:
df.to_csv('bond_ts_9Feb.csv')

These bond prices were scraped on Feb 9th to get the latest historical close prices. To keep things simple, all available historical close prices were scraped from the **1 month chart**. Prices between **16th Jan - 27th Jan** will then be used. This forms our period of 10 days of data. The index of the dataframe is how we refer to a bond. *"CAN 1.50 Jun 23"* refers to the Canadian Government Bond with a maturity in June 2023 and a coupon of 1.50.

In [63]:
df.drop(['2/9', '2/8', '2/7', '2/6', '2/3', '2/2', '2/1', '1/31',
       '1/30', '1/13', '1/12', '1/11', '1/10'], axis=1, inplace=True)
df.head()

,coupon,yield,maturityDate,ISIN,issuePrice,issueDate,1/27,1/26,1/25,1/24,1/23,1/20,1/19,1/18,1/17,1/16
CAN 1.50 Jun 23,1.50,4.55,2023-06-01,CA135087A610,97.98,2012-07-30,99.00,99.00,99.00,98.98,98.96,98.95,98.95,98.94,98.92,98.92
CAN 2.50 Jun 24,2.50,4.33,2024-06-01,CA135087B451,98.99,2013-07-02,97.96,98.07,98.07,98.01,98.02,98.05,98.11,98.17,98.07,98.08
CAN 2.25 Jun 25,2.25,3.68,2025-06-01,CA135087D507,98.85,2014-06-30,97.35,97.56,97.54,97.39,97.43,97.51,97.68,97.70,97.45,97.45
CAN 1.75 Mar 23,1.75,NaN,2023-03-01,CA135087H490,99.56,2017-10-06,99.79,99.78,99.77,99.76,99.75,99.74,99.72,99.72,99.70,99.71
CAN 2.25 Mar 24,2.25,4.49,2024-03-01,CA135087J546,98.99,2018-10-05,97.84,97.92,97.92,97.87,97.86,97.87,97.92,97.95,97.87,97.93


#### Selecting 10 Bonds
Now, 10 bonds which will be chosen to form our new dataframe to construct 0-5 year yield and spot curves:

* CAN 0.25 Feb 2023
* CAN 1.50 Jun 2023
* CAN 0.50 Nov 2023
* CAN 2.50 Jun 2024
* CAN 3.00 Nov 2024
* CAN 2.25 Jun 2025
* CAN 0.50 Sep 2025
* CAN 0.25 Mar 2026
* CAN 1.00 Sep 2026
* CAN 1.00 Jun 2027

These 10 bonds are chosen as they have maturity dates that are evenly distributed to plot the "0-5 year" yield & spot curves. Furthermore, they have coupons that are not too different. This forms a good basis and consistency for easier comparison of yields.

## 2.2 Empirical Questions

### Q4a: Calculating each of the 10 selected bond's yield

### Q4a: Plotting 5-year yield curve corresponding to each day of data superimposed on top of each other